# <img src="../assets/favicon.png" alt="Logo" width="50"/> **CHardy Tecno Store**

---

# ETL - Ventas

## 1. Extracción.

### 1.1. importamos los DataFrame de Ventas por año y los unimos.

In [1]:
import pandas as pd
import os

# Carpeta donde están los archivos
data_path = "data/raw/"
# Rango de años
anios = range(2018, 2025)

dfs = []

for anio in anios:
    parquet_file = os.path.join(data_path, f"fact_ventas_{anio}.parquet")
    csv_file = os.path.join(data_path, f"fact_ventas_{anio}.csv")
    
    try:
        if os.path.exists(parquet_file):
            df = pd.read_parquet(parquet_file)
            print(f"✔️ Archivo parquet cargado: {parquet_file}")
        elif os.path.exists(csv_file):
            df = pd.read_csv(csv_file)
            print(f"✔️ Archivo CSV cargado: {csv_file}")
        else:
            print(f"⚠️ No se encontró archivo para el año {anio}")
            continue
        
        # Agregamos columna de trazabilidad
        df["anio_origen"] = anio
        # Guardamos en lista
        dfs.append(df)
    
    except Exception as e:
        print(f"❌ Fallo al cargar {anio}: {e}")

# Concatenamos todos los dataframes
if dfs:
    df_final = pd.concat(dfs, ignore_index=True)
    print(f"[INFO] Dataset final armado con {len(df_final)} registros de {len(dfs)} años")
else:
    df_final = pd.DataFrame()
    print("[INFO] No se cargaron datos")

df = df_final

✔️ Archivo CSV cargado: data/raw/fact_ventas_2018.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2019.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2020.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2021.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2022.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2023.csv
✔️ Archivo CSV cargado: data/raw/fact_ventas_2024.csv
[INFO] Dataset final armado con 838945 registros de 7 años


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838945 entries, 0 to 838944
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   transaction_id               838945 non-null  int64  
 1   fecha                        838945 non-null  object 
 2   año                          838945 non-null  int64  
 3   mes                          838945 non-null  int64  
 4   cliente_id                   838945 non-null  int64  
 5   producto_id                  838945 non-null  int64  
 6   canal_id                     838945 non-null  int64  
 7   sucursal_id                  466720 non-null  float64
 8   cantidad                     838945 non-null  int64  
 9   precio_unitario_ars_nominal  838945 non-null  float64
 10  monto_venta_ars_nominal      838945 non-null  float64
 11  monto_venta_ars_real_2018    838945 non-null  float64
 12  es_evento                    838945 non-null  int64  
 13 

In [3]:
df.head()

,transaction_id,fecha,año,mes,cliente_id,producto_id,canal_id,sucursal_id,cantidad,precio_unitario_ars_nominal,monto_venta_ars_nominal,monto_venta_ars_real_2018,es_evento,anio_origen
0,1,2018-01-01,2018,1,5739,3,2,2.0,3,7944.48,23833.45,23833.45,0,2018
1,2,2018-01-01,2018,1,11442,22,2,15.0,2,3984.98,7969.96,7969.96,0,2018
2,3,2018-01-01,2018,1,9441,11,2,7.0,1,19582.89,19582.89,19582.89,0,2018
3,4,2018-01-01,2018,1,9808,24,2,25.0,3,2957.86,8873.58,8873.58,0,2018
4,5,2018-01-01,2018,1,13384,22,2,13.0,2,3927.82,7855.63,7855.63,0,2018


## 2. Transformación

In [4]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['sucursal_id'] = df['sucursal_id'].astype('Int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838945 entries, 0 to 838944
Data columns (total 14 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   transaction_id               838945 non-null  int64         
 1   fecha                        838945 non-null  datetime64[ns]
 2   año                          838945 non-null  int64         
 3   mes                          838945 non-null  int64         
 4   cliente_id                   838945 non-null  int64         
 5   producto_id                  838945 non-null  int64         
 6   canal_id                     838945 non-null  int64         
 7   sucursal_id                  466720 non-null  Int64         
 8   cantidad                     838945 non-null  int64         
 9   precio_unitario_ars_nominal  838945 non-null  float64       
 10  monto_venta_ars_nominal      838945 non-null  float64       
 11  monto_venta_ars_real_2018 

### 2.1. Busqueda de Duplicados

In [5]:
# Filtrar duplicados (todas las columnas)
df_duplicados = df[df.duplicated()] # Busca filas duplicadas o sea dos filas exactamente iguales
print("Registros Duplicados:", df_duplicados)
# Filtrar duplicados (columnas críticas)
df_duplicados_id = df[df.duplicated(subset=["transaction_id"], keep=False)]
print("transaction_id (duplicados):\n", df_duplicados_id)


Registros Duplicados: Empty DataFrame
Columns: [transaction_id, fecha, año, mes, cliente_id, producto_id, canal_id, sucursal_id, cantidad, precio_unitario_ars_nominal, monto_venta_ars_nominal, monto_venta_ars_real_2018, es_evento, anio_origen]
Index: []
transaction_id (duplicados):
 Empty DataFrame
Columns: [transaction_id, fecha, año, mes, cliente_id, producto_id, canal_id, sucursal_id, cantidad, precio_unitario_ars_nominal, monto_venta_ars_nominal, monto_venta_ars_real_2018, es_evento, anio_origen]
Index: []


### 2.2. Busqueda Valores Null (NaN)

In [6]:
print(df.isnull().sum()) # Muestra valores nulos (valor cero no es nulo)

transaction_id                      0
fecha                               0
año                                 0
mes                                 0
cliente_id                          0
producto_id                         0
canal_id                            0
sucursal_id                    372225
cantidad                            0
precio_unitario_ars_nominal         0
monto_venta_ars_nominal             0
monto_venta_ars_real_2018           0
es_evento                           0
anio_origen                         0
dtype: int64


Esos NaN en sucursal_id podrian ser ventas online, asi que lo verificaremos

In [7]:
ventas_online = df[df["sucursal_id"].isna() & (df["canal_id"] == 1)]
print(ventas_online.head())

# Contar valores nulos en la columna 'sucursal_id'
nulos_sucursal = df["sucursal_id"].isna().sum()
print("Nulos en sucursal_id: ", nulos_sucursal)
print("Ventas Online:     -> ", ventas_online.shape[0])


    transaction_id      fecha   año  mes  cliente_id  producto_id  canal_id  \
13              14 2018-01-01  2018    1        7099           20         1   
16              17 2018-01-01  2018    1       13640            4         1   
31              32 2018-01-01  2018    1         563           13         1   
33              34 2018-01-01  2018    1       14443           26         1   
37              38 2018-01-01  2018    1        5895           13         1   

    sucursal_id  cantidad  precio_unitario_ars_nominal  \
13         <NA>         1                      4510.93   
16         <NA>         1                      2566.72   
31         <NA>         1                     14561.97   
33         <NA>         2                      1940.66   
37         <NA>         2                     15291.14   

    monto_venta_ars_nominal  monto_venta_ars_real_2018  es_evento  anio_origen  
13                  4510.93                    4510.93          0         2018  
16            

In [8]:
# Reemplazar NaN en sucursal_id por 0 que agregue en el etl de sucursales para ventas online
df.loc[df["sucursal_id"].isna(), "sucursal_id"] = 0

print("Registros Modificados: ", (df["sucursal_id"] == 0).sum())

Registros Modificados:  372225


## 3. Carga

### 3.1. Guardamos el DF transformado y limpio.

In [9]:
import os

filename = "data/processed/fact_ventas.parquet"

try:
    df.to_parquet(filename, engine="fastparquet")
    if os.path.exists(filename):
        print(f"✔️ Dataset guardado con éxito en: {filename}")
    else:
        print("⚠️ No se encontró el archivo después de guardar.")
except Exception as e:
    print(f"❌ Error al guardar el dataset: {e}")

✔️ Dataset guardado con éxito en: data/processed/fact_ventas.parquet
